In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sb
from sklearn import metrics
from sklearn.model_selection import train_test_split

sb.set_theme(style="whitegrid", palette="muted", font_scale=1.5)
drive.mount('/content/drive')

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
reshaped_segments = np.asarray(
    segments, dtype = np.float32).reshape(-1, N_time_steps, N_features)

reshaped_segments.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    reshaped_segments, labels, test_size=0.2
)